In [1]:
from rtbm.riemann_theta.riemann_theta import RiemannTheta

import pandas as pd
import numpy as np

from sklearn import linear_model
from sklearn.metrics import confusion_matrix, classification_report

import theano

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Reshape  
from keras.optimizers import SGD

from sklearn.preprocessing import LabelBinarizer

import time

from rtbm import RTBM, minimizer

import rtbm.layers as layers
import rtbm.model as mdl

from rtbm.costfunctions import mse
from rtbm.activations import sigmoid, linear

Using Theano backend.
Process PoolWorker-1:
Process PoolWorker-6:
Process PoolWorker-8:
Traceback (most recent call last):
Process PoolWorker-3:
Traceback (most recent call last):
Process PoolWorker-7:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-2:
Process PoolWorker-4:
Traceback (most recent call last):
Process PoolWorker-5:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/proc

In [2]:
# Load MNIST dataset
MNIST_train = pd.read_csv('~/data/mnist_train.csv', delimiter=",",header=None).values
MNIST_test  = pd.read_csv('~/data/mnist_test.csv', delimiter=",",header=None).values

# Prepare data (normalized onto [0,1])
Y_train = MNIST_train[0:10000,0]
X_train = MNIST_train[0:10000,1:]/255.0

Y_test = MNIST_test[:,0]
X_test = MNIST_test[:,1:]/255.0

enc = LabelBinarizer()
enc.fit(np.diag([1,1,1,1,1,1,1,1,1,1]))
enc.classes_ = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
T=enc.transform(Y_train)

# Keras (500 linear + 100 linear + 1 linear + MSE)

In [ ]:
model = Sequential() 

model.add(Dense(500,  input_dim=784))
model.add(Activation('linear'))
#model.add(Dense(100,  input_dim=784))
#model.add(Activation('linear'))
model.add(Dense(output_dim=1))
model.add(Activation('linear'))


sgd = SGD(lr=0.001)

tic = time.clock()

model.compile(loss='mse', optimizer=sgd)

toc = time.clock()

print("Compile time: ",toc-tic)

tic = time.clock()

model.fit(X_train, Y_train, batch_size=10000, nb_epoch=100, validation_data=None, shuffle=False, verbose=1)  
toc = time.clock()

print("Run time: ",toc-tic)

In [ ]:
# On train set
P=np.abs(np.round(np.real(model.predict(X_train)))).flatten()


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
# On test set
P=np.abs(np.round(np.real(model.predict(X_test)))).flatten()


print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

## With Theta and SGD


In [ ]:
M = mdl.Model()
M.add(layers.NonLinear(784,500,linear()))
#M.add(layers.NonLinear(500,100,linear()))
M.add(layers.NonLinear(500,1,linear()))

minim = minimizer.SGD()
sol=minim.train(mse(), M, np.transpose(X_train), Y_train.reshape(1,len(Y_train)), lr=0.001, maxiter=100, batch_size=10000, log_step=10)


In [ ]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))

print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))

print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

# Keras (200 sigmoids + 10 sigmoids + 1 linear + MSE)

In [ ]:
model = Sequential() 

model.add(Dense(10,  input_dim=784))
model.add(Activation('sigmoid'))
#model.add(Dense(10))
#model.add(Activation('sigmoid'))
model.add(Dense(output_dim=1))
model.add(Activation('linear'))


sgd = SGD(lr=0.1)

tic = time.clock()

model.compile(loss='mse', optimizer=sgd)

toc = time.clock()

print("Compile time: ",toc-tic)

tic = time.clock()

model.fit(X_train, Y_train, batch_size=10000, nb_epoch=200, validation_data=None, shuffle=False, verbose=0)  
toc = time.clock()

print("Run time: ",toc-tic)

In [ ]:
# On train set
P=np.abs(np.round(np.real(model.predict(X_train)))).flatten()


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
# On test set
P=np.abs(np.round(np.real(model.predict(X_test)))).flatten()


print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

# With Theta and SGD

In [ ]:
M = mdl.Model()
M.add(layers.NonLinear(784,10,sigmoid()))
#M.add(layers.NonLinear(200,10,sigmoid()))
M.add(layers.Linear(10,1))

minim = minimizer.SGD()
sol=minim.train(mse(), M, np.transpose(X_train), Y_train.reshape(1,len(Y_train)), lr=0.1, maxiter=200, batch_size=10000, log_step=10)


In [ ]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))

print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))


# Keras (200 sigmoids + 10 Softmax + MSE)¶

In [27]:
model = Sequential() 

model.add(Dense(200,  input_dim=784))
model.add(Activation('sigmoid'))
model.add(Dense(10,  input_dim=784))
model.add(Activation('softmax'))


sgd = SGD(lr=0.001)

tic = time.clock()

model.compile(loss='mse', optimizer=sgd)

toc = time.clock()

print("Compile time: ",toc-tic)

tic = time.clock()

model.fit(X_train, T, batch_size=1000, nb_epoch=100, validation_data=None, shuffle=False, verbose=0)  
toc = time.clock()

print("Run time: ",toc-tic)

('Compile time: ', 0.08836900000005699)
('Run time: ', 60.80085400000007)


In [28]:
# On train set
P=np.argmax(model.predict(X_train),axis=1)

print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1001
          1       0.00      0.00      0.00      1127
          2       0.00      0.00      0.00       991
          3       0.00      0.00      0.00      1032
          4       0.23      0.18      0.20       980
          5       0.00      0.00      0.00       863
          6       0.00      0.00      0.00      1014
          7       0.14      0.01      0.02      1070
          8       0.09      0.92      0.17       944
          9       0.00      0.00      0.00       978

avg / total       0.05      0.10      0.04     10000

[[   0    0    0    0  225    0    0   11  765    0]
 [   0    0    0    0    0    0    0    0 1127    0]
 [   0    0    0    0   33    0    0    6  952    0]
 [   0    0    0    0   25    0    0    8  999    0]
 [   0    0    0    0  172    0    0    4  804    0]
 [   0    0    0    0   73    0    0    1  789    0]
 [   0    0    0    0   89    0    0    4  

In [29]:
# On test set
P=np.argmax(model.predict(X_test),axis=1)

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       980
          1       0.00      0.00      0.00      1135
          2       0.00      0.00      0.00      1032
          3       0.00      0.00      0.00      1010
          4       0.18      0.15      0.16       982
          5       0.00      0.00      0.00       892
          6       0.00      0.00      0.00       958
          7       0.07      0.01      0.01      1028
          8       0.09      0.88      0.17       974
          9       0.00      0.00      0.00      1009

avg / total       0.03      0.10      0.03     10000

[[   0    0    0    0  262    0    0    5  713    0]
 [   0    0    0    0    0    0    0    0 1135    0]
 [   0    0    0    0   37    0    0   11  984    0]
 [   0    0    0    0   29    0    0    9  972    0]
 [   0    0    0    0  148    0    0    2  832    0]
 [   0    0    0    0   72    0    0    9  811    0]
 [   0    0    0    0  106    0    0    4  

## theta

In [30]:
M = mdl.Model()
M.add(layers.NonLinear(784,200,sigmoid()))
M.add(layers.Linear(200,10))
M.add(layers.SoftMaxLayer(10))

minim = minimizer.SGD()
sol=minim.train(mse(), M, np.transpose(X_train), T.T, lr=0.001, maxiter=200, batch_size=1000, log_step=10)


('Batch C: ', 0.9096904861920732)
Iteration 0 in 0.75(s), cost = 0.908560
('Batch C: ', 0.9035433808556956)
Iteration 10 in 6.03(s), cost = 0.902852
('Batch C: ', 0.9022354225371491)
Iteration 20 in 11.33(s), cost = 0.901163
('Batch C: ', 0.9029663782921442)
Iteration 30 in 16.73(s), cost = 0.901145
('Batch C: ', 0.9023120790327643)
Iteration 40 in 21.99(s), cost = 0.900584
('Batch C: ', 0.9012368225864998)
Iteration 50 in 27.26(s), cost = 0.900238
('Batch C: ', 0.9000391550477479)
Iteration 60 in 32.46(s), cost = 0.899322
('Batch C: ', 0.9004836578809717)
Iteration 70 in 37.61(s), cost = 0.899173
('Batch C: ', 0.9004952675355563)
Iteration 80 in 42.83(s), cost = 0.898937
('Batch C: ', 0.8994279630427919)
Iteration 90 in 48.04(s), cost = 0.898765
('Batch C: ', 0.898792103776735)
Iteration 100 in 53.30(s), cost = 0.898219
('Batch C: ', 0.899715911597967)
Iteration 110 in 58.57(s), cost = 0.898208
('Batch C: ', 0.8989355617914094)
Iteration 120 in 63.82(s), cost = 0.898200
('Batch C: ', 

In [31]:
P=np.argmax(M.predict(np.transpose(X_train)),axis=0)

print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

             precision    recall  f1-score   support

          0       0.53      0.06      0.11      1001
          1       0.50      0.00      0.00      1127
          2       0.08      0.06      0.07       991
          3       0.14      0.00      0.01      1032
          4       0.09      0.44      0.15       980
          5       0.00      0.00      0.00       863
          6       0.05      0.04      0.05      1014
          7       0.04      0.02      0.02      1070
          8       0.00      0.00      0.00       944
          9       0.20      0.63      0.31       978

avg / total       0.17      0.12      0.07     10000

[[ 64   0   1   4 386   0  78 369   2  97]
 [  0   1 526   0 443   0 113   0   0  44]
 [  3   0  62   0 674   0  65   8   1 178]
 [ 30   0 122   3 518   2  92  15   1 249]
 [  3   1  17   1 434   0  70  15   0 439]
 [ 20   0  11   4 423   0  90  32   2 281]
 [  0   0  13   1 756   0  42   3   0 199]
 [  1   0  10   7 442   0  91  18   2 499]
 [  0   0  45   0

In [32]:
P=np.argmax(M.predict(np.transpose(X_test)),axis=0)


print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

             precision    recall  f1-score   support

          0       0.61      0.08      0.13       980
          1       0.00      0.00      0.00      1135
          2       0.09      0.06      0.07      1032
          3       0.18      0.00      0.01      1010
          4       0.10      0.48      0.17       982
          5       0.00      0.00      0.00       892
          6       0.06      0.05      0.05       958
          7       0.04      0.01      0.02      1028
          8       0.00      0.00      0.00       974
          9       0.19      0.65      0.29      1009

avg / total       0.13      0.13      0.07     10000

[[ 74   0   3   4 381   1  80 323   4 110]
 [  0   0 472   0 545   0  92   0   0  26]
 [  3   0  64   0 642   0  70  10   1 242]
 [ 14   0  79   4 517   0  95  12   7 282]
 [  1   0  17   1 467   0  47  11   0 438]
 [ 16   0  10   6 335   0 118  21   1 385]
 [  8   0  14   0 606   0  45   3   0 282]
 [  2   0  10   4 427   0  94  15   0 476]
 [  2   0  32   3